In [ ]:
from tespy.connections import HumidAirConnection, Connection, Ref
from tespy.components import Source, Sink, MovingBoundaryHeatExchanger
from tespy.networks import Network

from CoolProp.CoolProp import HAPropsSI

In [ ]:
nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar",
    heat="kW"
)

In [ ]:
def get_water_air_mixture_fractions_pTR(p, T, R):
    w = HAPropsSI("W", "P", p, "T", T, "R", R)
    return get_water_air_mixture_fractions_w(w)


def get_water_air_mixture_fractions_w(w):
    air = 1 / (1 + w)
    return {"air": air, "water": 1 - air}

In [ ]:
so = Source("source")
hex = MovingBoundaryHeatExchanger("heat exchanger")
si = Sink("sink")

so2 = Source("refrigerant source")
si2 = Sink("refrigerant sink")

a1 = Connection(so2, "out1", hex, "in2", label="a1")
a2 = Connection(hex, "out2", si2, "in1", label="a2")

c1 = Connection(so, "out1", hex, "in1", label="c1")
c2 = Connection(hex, "out1", si, "in1", label="c2")

nw.add_conns(a1, a2, c1, c2)

a1.set_attr(fluid={"NH3": 1}, m=2, x=0.3)
a2.set_attr(td_dew=5, T_dew=-20)

c1.set_attr(
    fluid0={"water": 0.0005, "air": 0.9995},#get_water_air_mixture_fractions_pTR(p=1e5, T=268.16, R=1),
    p=1,
    T=-5,
    mixing_rule="humidair",
    r=1,
    fluid_balance=True,
    h0=0
)
c2.set_attr(T=Ref(c1, 1, -5), h0=0)

hex.set_attr(dp1=0, dp2=0)

nw.solve("design")

In [ ]:
c2.set_attr(T=None)
c2.set_attr(T=-10)
nw.solve("design")

In [ ]:
c1.fluid.val

In [ ]:
c2.fluid.val

In [ ]:
nw.print_results()

In [ ]:
heat, T_hot, T_cold, _, _ = hex.calc_sections()

In [ ]:
from matplotlib import pyplot as plt


plt.plot(heat, T_hot)
plt.plot(heat, T_cold)